# Assignment 1

Q1

In [ ]:
def encrypt(text,a,b):
  new = ""
  for i in text:
    if i.isalpha():
      if i.isupper():
        new += chr(((a * (ord(i) - ord('A')) + b) % 26) + ord('A'))
      else:
        new += chr(((a * (ord(i) - ord('a')) + b) % 26) + ord('a'))
    else:
      new += i
  return new

def decrypt(text, a, b):
   decrypted_text = ""
   a_inv = pow(a, -1, 26)
   for char in text:
      if char.isalpha():
         if char.isupper():
            decrypted_text += chr(((a_inv * (ord(char) - ord('A') - b)) % 26) + ord('A'))
         else:
            decrypted_text += chr(((a_inv * (ord(char) - ord('a') - b)) % 26) + ord('a'))
      else:
         decrypted_text += char
   return decrypted_text

text="HELLO"
a=5
b=8
z = encrypt(text,a,b)
x = decrypt(z,a,b)
print(z)
print(x)

RCLLA
HELLO


Q2

In [ ]:
def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def affine_decrypt(ciphertext, a, b):
    a_inv = mod_inverse(a, 26)
    if a_inv is None:
        raise ValueError(f"No modular inverse for a = {a} under modulo 26.")

    decrypted_text = ""
    for char in ciphertext:
        if char.isalpha():
            offset = ord('A') if char.isupper() else ord('a')
            decrypted_text += chr(((a_inv * (ord(char) - offset - b)) % 26) + offset)
        else:
            decrypted_text += char
    return decrypted_text

def determine_keys(ciphertext):
    x1, y1 = ord('E') - ord('A'), ord('S') - ord('A')
    x2, y2 = ord('T') - ord('A'), ord('X') - ord('A')
    a = ((y1 - y2) * mod_inverse(x1 - x2, 26)) % 26
    b = (y1 - a * x1) % 26
    return a, b

ciphertext = "FMXVEDKAPHEERBNDKXRSREFMORUSDKDVSHVUFEDKAPRKDLYEVLRHHRH"
a, b = determine_keys(ciphertext)
print(f"Determined keys: a = {a}, b = {b}")
print(f"Decrypted text: {affine_decrypt(ciphertext, a, b)}")


Determined keys: a = 9, b = 8
Decrypted text: RMTNOLGCVXOOBFPLGTBEBORMSBKELGLNEXNKROLGCVBGLJWONJBXXBX


Q3

In [ ]:
def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def encrypt_affine(text, a, b):
    result = ""
    for char in text:
        if char.isalpha():
            offset = ord('A') if char.isupper() else ord('a')
            result += chr(((a * (ord(char) - offset) + b) % 26) + offset)
        else:
            result += char
    return result

def decrypt_affine(text, a, b):
    a_inv = mod_inverse(a, 26)
    if a_inv is None:
        raise ValueError(f"No modular inverse for a = {a} under modulo 26.")
    result = ""
    for char in text:
        if char.isalpha():
            offset = ord('A') if char.isupper() else ord('a')
            result += chr(((a_inv * (ord(char) - offset - b)) % 26) + offset)
        else:
            result += char
    return result

def apply_salt(text, salt):
    result = ""
    for char in text:
        if char.isalpha():
            offset = ord('A') if char.isupper() else ord('a')
            result += chr(((ord(char) - offset + salt) % 26) + offset)
        else:
            result += char
    return result

def remove_salt(text, salt):
    result = ""
    for char in text:
        if char.isalpha():
            offset = ord('A') if char.isupper() else ord('a')
            result += chr(((ord(char) - offset - salt) % 26) + offset)
        else:
            result += char
    return result

plain_text = "HELLO"
salt = 2
a, b = 5, 8

salted_text = apply_salt(plain_text, salt)
encrypted_text = encrypt_affine(salted_text, a, b)
decrypted_text = decrypt_affine(encrypted_text, a, b)
original_text = remove_salt(decrypted_text, salt)

print(f"Modified Plaintext with salt: {salted_text}")
print(f"Encrypted text: {encrypted_text}")
print(f"Decrypted text: {original_text}")


Modified Plaintext with salt: JGNNQ
Encrypted text: BMVVK
Decrypted text: HELLO


Q4

In [ ]:
from math import gcd
from collections import Counter

def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def affine_decrypt(text, a, b):
    a_inv = mod_inverse(a, 26)
    if a_inv is None:
        return None
    result = ""
    for char in text:
        if char.isalpha():
            result += chr(((a_inv * (ord(char) - ord('A') - b)) % 26) + ord('A'))
        else:
            result += char
    return result

def remove_salt(text, salt):
    result = ""
    for char in text:
        if char.isalpha():
            result += chr(((ord(char) - ord('A') - salt) % 26) + ord('A'))
        else:
            result += char
    return result

def is_coprime(a, m):
    return gcd(a, m) == 1

def find_keys(ciphertext):
    freq = Counter(ciphertext.replace(" ", ""))
    common_letters = ['E', 'T', 'A', 'O', 'I', 'N']
    for cipher_char1, _ in freq.most_common(5):
        for cipher_char2, _ in freq.most_common(5):
            if cipher_char1 == cipher_char2:
                continue
            for plain_char1 in common_letters:
                for plain_char2 in common_letters:
                    if plain_char1 == plain_char2:
                        continue
                    y1, y2 = ord(cipher_char1) - ord('A'), ord(cipher_char2) - ord('A')
                    x1, x2 = ord(plain_char1) - ord('A'), ord(plain_char2) - ord('A')
                    delta_y = (y1 - y2) % 26
                    delta_x = (x1 - x2) % 26
                    if mod_inverse(delta_x, 26):
                        a = (delta_y * mod_inverse(delta_x, 26)) % 26
                        if is_coprime(a, 26):
                            b = (y1 - a * x1) % 26
                            return a, b
    return None, None

def find_salt(ciphertext, a, b):
    salted_text = affine_decrypt(ciphertext.replace(" ", ""), a, b)
    if not salted_text:
        return None, None
    for salt in range(26):
        plain_text = remove_salt(salted_text, salt)
        if is_valid_text(plain_text):
            return salt, plain_text
    return None, None

def is_valid_text(text):
    common_words = ["the", "and", "is", "of", "to", "in", "that", "it"]
    return any(word in text.lower() for word in common_words)

def solve_affine_cipher(ciphertext):
    a, b = find_keys(ciphertext)
    if a is None or b is None:
        return "Failed to determine keys."
    salt, plain_text = find_salt(ciphertext, a, b)
    if salt is None or plain_text is None:
        return "Failed to determine salt or decrypt the text."
    return f"Keys: a = {a}, b = {b}\nSalt: {salt}\nPlaintext: {plain_text}"

ciphertext = "TBRDOB RZWVUL TDMABRY VT H HPHCVUL TDMABRY"
print(solve_affine_cipher(ciphertext))


Failed to determine keys.
